# Amazon Bedrock と Amazon Kendra を用いた Advanced RAG 構築

本ノートブックでは、Amazon Bedrock を生成器、Amazon Kendra を検索器として RAG システムを構築します。下記の順番で、まずは抽象度低く Naive RAG を構築し、動作を理解してから LangChain や LangGraph を用いた Advanced RAG システムの構築を体験します。

1. AWS SDK for Python (boto3) を用いて Naive RAG を構築
2. LangChain を用いて Naive RAG を構築
3. LangGraph を用いて Naive RAG & Advanced RAG を構築

## 1. 事前準備

Ctrl + \` で Code Editor (OSS 版 VS Code) のターミナルを開き、以下のコマンドを実行してください。

```bash:
python3 -m venv .venv
source .venv/bin/activate
pip install -U pip
pip install -r requirements.txt
```

## 2. boto3 を直接用いて RAG を構築

### 必要なモジュールのインポートと boto3 クライアントの準備

AWS SDK for Python (boto3) 等のモジュールをインポートし、事前準備を行います。
ノートブックの各セルを実行するには `Shift + Enter` を押してください。

In [31]:
import re  # 正規表現
import json
from pprint import pprint

from IPython.display import (
    display,
    Markdown,
)

import boto3
from botocore.exceptions import ClientError

# Amazon Kendra と Amazon Bedrock のクライアント
kendra = boto3.client("kendra", region_name="us-west-2")
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-west-2")

# Amazon Bedrock で利用する Claude 3 Haiku のモデル ID
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

accept = "application/json"
content_type = "application/json"

#### Amazon Kendra のインデックス ID を入力

ワークショップの手順書のタイトルをクリックしてイベントのダッシュボードを開き、パラメーターのリスト (Event Outputs) にある `KendraIndexID` の値を以下にコピーしてください。

In [32]:
# TODO: 自身の環境に合わせて書き換える
kendra_index_id = "1e74d846-4929-49a4-83bb-00f4c97baf2a"  # 36 文字

### まずは Amazon Bedrock に直接問いかけてみる

`query_text` に好きな質問を入れて Claude 3 Haiku に問いかけてみましょう。

In [6]:
# ユーザーの質問
query_text = "Amazon Bedrockのモデル評価機能ではどのようなことができますか？"

In [43]:
# Amazon Bedrock に送るリクエストのボディー
body = json.dumps({
    "max_tokens": 1024,  # 最大出力トークン数
    "messages": [
        {"role": "user", "content": query_text},
        # {"role": "assistant", "content": "いい質問ですね。"},
        # のように user -> assistant -> user -> ... の会話形式にすることも可能
    ],
    # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html
    "anthropic_version": "bedrock-2023-05-31",  # 固定
    "temperature": 0,  # 出力のランダム度
})

try:
    # Amazon Bedrock の InvokeModel API を実行
    response_bedrock = bedrock_runtime.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type,
    )
    response_body = json.loads(response_bedrock.get("body").read())
    display(Markdown(response_body.get("content")[0]["text"]))
except ClientError as error:
    raise error

Amazon Bedrockのモデル評価機能には以下のような主な機能があります:

1. モデルの性能評価:
   - 精度、再現率、F1スコアなどの指標を使ってモデルの性能を評価できます。
   - 様々なデータセットを使ってモデルのパフォーマンスを比較できます。

2. モデルの解釈性分析:
   - モデルの内部動作を可視化して、どのような特徴が重要かを理解できます。
   - 特徴の重要度を定量的に分析できます。

3. モデルのロバスト性評価:
   - 入力データに対するモデルの耐性を評価できます。
   - 攻撃的な入力に対するモデルの挙動を分析できます。

4. モデルのデプロイ準備:
   - モデルの最適化や軽量化を行い、本番環境での利用を支援できます。
   - モデルのリソース消費や推論時間などの指標を確認できます。

5. モデルのモニタリング:
   - 本番環境でのモデルの振る舞いを監視し、異常を検知できます。
   - 継続的な学習や再トレーニングのタイミングを判断できます。

このように、Amazon Bedrockのモデル評価機能は、機械学習モデルの開発から運用までのライフサイクル全体をサポートしています。

#### InvokeModel API の理解を深める

なお、`response_bedrock` の中身は以下のようになっており、`body` キーには `StreamingBody` の形で出力が格納されています。

In [35]:
response_bedrock

{'ResponseMetadata': {'RequestId': '73b09d92-c520-471d-a929-25a68dda4f0e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 27 May 2024 08:18:02 GMT',
   'content-type': 'application/json',
   'content-length': '1755',
   'connection': 'keep-alive',
   'x-amzn-requestid': '73b09d92-c520-471d-a929-25a68dda4f0e',
   'x-amzn-bedrock-invocation-latency': '5548',
   'x-amzn-bedrock-output-token-count': '479',
   'x-amzn-bedrock-input-token-count': '31'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x1181f2740>}

`response_body` は JSON であり、`content` キーの中に辞書のリスト形式で出力文が格納されます。他にも例えば `usage` キーで入力と出力のトークン数を確認でき、実際にどの程度の料金がかかったかを計算することができます。

In [36]:
pprint(response_body)

{'content': [{'text': 'Amazon Bedrockのモデル評価機能には以下のような主な機能があります:\n'
                      '\n'
                      '1. モデルの性能評価:\n'
                      '   - '
                      '分類、回帰、クラスタリングなどのタスクに対するモデルの精度、再現率、F1スコアなどの指標を評価できます。\n'
                      '   - 複数のモデルを比較して、最適なモデルを選択することができます。\n'
                      '\n'
                      '2. モデルの解釈性分析:\n'
                      '   - モデルの内部構造や重要な特徴を可視化することで、モデルの動作を理解しやすくできます。\n'
                      '   - 特徴の重要度を確認したり、モデルの予測に影響を与える要因を特定できます。\n'
                      '\n'
                      '3. モデルのデータ依存性分析:\n'
                      '   - トレーニングデータの特性が予測結果に与える影響を分析できます。\n'
                      '   - データの偏りや欠落が及ぼすモデルの性能への影響を評価できます。\n'
                      '\n'
                      '4. モデルのロバスト性評価:\n'
                      '   - 入力データに対するモデルの耐性を確認できます。\n'
                      '   - 攻撃的な入力に対してモデルがどのように振る舞うかを分析できます。\n'
                      '\n'
                      '5. モデルのデプロイ準備:\n'
                      '   - モデルの出力

### Amazon Kendra で検索する

AWS のインテリジェント検索サービスである Amazon Kendra の検索を試してみましょう。

Kendra でドキュメントを検索するには [Query API](https://docs.aws.amazon.com/ja_jp/kendra/latest/APIReference/API_Query.html) と [Retrieve API](https://docs.aws.amazon.com/ja_jp/kendra/latest/APIReference/API_Retrieve.html) のふたつの方法があります。今回は、RAG との親和性が高い Retrieve API を用いることで、データソースの中のドキュメントから、ユーザーの質問内容に関連する抜粋部分を抽出し、生成 AI への入力としていきます。

なお、Retrieve API では `PageSize` (デフォルト値は10件) を超える抜粋部分が抽出された際、レスポンスがページングされ、追加の `PageSize` 分ごとに結果を再取得していく必要があります。本ワークショップで RAG を実装する際には、最初の10件の抜粋のみを後続の処理に渡していくこととします。

In [13]:
response_kendra = kendra.retrieve(
    QueryText=query_text,  # 検索クエリ
    IndexId=kendra_index_id,  # Kendra Index ID
    AttributeFilter={
        "EqualsTo": {
            "Key": "_language_code",
            "Value": {
                "StringValue": "ja",  # 日本語ドキュメントを検索 (default: en)
            }
        }
    },
    #PageNumber=1,
    PageSize=3,  # 一度に返すドキュメント抜粋の数 (default: 10)
)

print("Query Text:", query_text)
print("Number of Retrieved Items:", len(response_kendra["ResultItems"]))

for ii, item in enumerate(response_kendra["ResultItems"]):
    print("-" * 30)
    print("Item", ii + 1)
    print("DocumentTitle:", item["DocumentTitle"])
    print("Content")
    pprint(item["Content"])

Query Text: Amazon Bedrockのモデル評価機能ではどのようなことができますか？
Number of Retrieved Items: 3
------------------------------
Item 1
DocumentTitle: モデルカスタマイズに関するガイドライン - Amazon Bedrock
Content
('AWSドキュメントAmazon Bedrockユーザーガイド Amazon Titan Text G1 - Express '
 '翻訳は機械翻訳により提供されています。 提供された翻訳内容と英語版の間で齟齬、不一致または矛盾がある場合、英語版が優先します。 '
 'モデルカスタマイズに関するガイドライン モデルをカスタマイズする理想的なパラメータは、データセットと、モデルが対象とするタスクによって異なります。 '
 '値をいろいろ試して、どのパラメータがお客様自身のケースで最も適切に機能するかを確認する必要があります。 '
 '提供された翻訳内容と英語版の間で齟齬、不一致または矛盾がある場合、英語版が優先します。 モデルカスタマイズに関するガイドライン '
 'モデルをカスタマイズする理想的なパラメータは、データセットと、モデルが対象とするタスクによって異なります。 '
 '値をいろいろ試して、どのパラメータがお客様自身のケースで最も適切に機能するかを確認する必要があります。 '
 '参考までに、モデル評価ジョブを実行してモデルを評価してください。 詳細については、「モデル評価」を参照してください。 このトピックでは、Amazon '
 'Titan Text G1 - Express モデルをカスタマイズするための基準となるガイドラインと推奨値を示します。 '
 '他のモデルについては、プロバイダーのドキュメントを確認してください。 '
 'ファインチューニングジョブの送信時に生成される出力ファイルに含まれるトレーニングと検証のメトリクスを使用して、パラメータを調整します。')
------------------------------
Item 2
DocumentTitle: モデル評価ジョブの結果が Amazon S3 にどのように保存されるかを理解する - Amazon Bedrock
Conten

#### Retrieve API の理解を深める

Retrieve API の `response_kendra` の中にある `ResultItems` キーに、検索で得られたドキュメントの抜粋の情報が格納されています。

In [11]:
response_kendra

{'QueryId': '3797b6a2-1bf7-4002-8a3d-1fd2f695500c',
 'ResultItems': [],
 'ResponseMetadata': {'RequestId': '3ae704f9-7599-457b-a5a6-a73eb2db2ed3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3ae704f9-7599-457b-a5a6-a73eb2db2ed3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '67',
   'date': 'Mon, 27 May 2024 07:06:50 GMT'},
  'RetryAttempts': 0}}

`ResultItems` の各項目の中身は [`RetrieveResultItem`](https://docs.aws.amazon.com/kendra/latest/APIReference/API_RetrieveResultItem.html) オブジェクトです。`RetrieveResulltItem` は以下の情報を含みます。

- `Content`: 検索でヒットしたドキュメントの抜粋部分のテキスト
- `DocumentAttributes`: ドキュメントのソース URI や抜粋のページ番号などの情報
- `DocumentId`: ドキュメントの ID
- `DocumentTitle`: ドキュメントのタイトル
- `DocumentURI`: ドキュメントの URI
- `Id`: ドキュメントの抜粋のユニークな ID
- `ScoreAttribute`: クエリとの関連度合いのスコア (日本語は未対応で `NOT_AVAILABLE` が返る)

In [12]:
pprint(response_kendra["ResultItems"][0])

IndexError: list index out of range

以下では、Amazon Kendra の Retrieve API で得られたドキュメントの抜粋を Amazon Bedrock に送るプロンプトに含めることで RAG を実現していきます。

### Amazon Bedrock と Kendra を組み合わせて RAG を実行する

ここまでの情報だけで RAG を実現することができます。LLM に与えるプロンプトの中に Kendra で取得したドキュメントの抜粋を入れ込み、質問に答えてもらうように指示します。

#### RAG 用のプロンプト

サンプルのプロンプトテンプレートは以下の通りです。ここでは、`<excerpt>` (抜粋) タグ内にドキュメントの抜粋を記載し、`<query>` タグ内に質問文を記載しています。タグ名は固定ではなく、任意の値を設定できます。

XML タグを理解するように学習されているのは Claude の特徴であり、XML タグを利用することで構造化されたプロンプトを簡単に構築することができます。詳しくは [Anthropic のプロンプトエンジニアリングガイド](https://docs.anthropic.com/ja/docs/use-xml-tags)を参照してください。

In [11]:
prompt = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
"""

for item in response_kendra["ResultItems"]:
    prompt += f"<excerpt>{item['Content']}</excerpt>\n"

prompt += f"""</excerpts>

<excerpts>タグ内の情報を参考にして、<query>タグ内のユーザーの質問に答えてください。

<query>
{query_text}
</query>
"""

print(prompt)


あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
<excerpt>print( f"Finished generating text with your provisioned custom model {model_id}.") if __name__ == "__main__": main() モデルカスタマイズに関するガイドライン モデルをカスタマイズする理想的なパラメータは、データセットと、モデルが対象とするタスクに よって異なります。 値をいろいろ試して、どのパラメータがお客様自身のケースで最も適切に機能す るかを確認する必要があります。 参考までに、モデル評価ジョブを実行してモデルを評価してくださ い。 詳細については、「モデル評価」を参照してください。 このトピックでは、Amazon Titan Text G1 - Express モデルをカスタマイズするための基準となるガ イドラインと推奨値を示します。 他のモデルについては、プロバイダーのドキュメントを確認してく ださい。 ファインチューニングジョブの送信時に生成される出力ファイルに含まれるトレーニングと検証のメ トリクスを使用して、パラメータを調整します。 出力を書き込んだ Amazon S3 バケットでこれらの ファイルを検索するか、GetCustomModelオペレーションを使用してください。</excerpt>
<excerpt>AWSドキュメントAmazon Bedrockユーザーガイド Amazon Titan Text G1 - Express 翻訳は機械翻訳により提供されています。 提供された翻訳内容と英語版の間で齟齬、不一致または矛盾がある場合、英語版が優先します。 モデルカスタマイズに関するガイドライン モデルをカスタマイズする理想的なパラメータは、データセットと、モデルが対象とするタスクによって異なります。 値をいろいろ試して、どのパラメータがお客様自身のケースで最も適切に機能するかを確認する必要があります。 提供された翻訳内容と英語版の間で齟齬、不一致または矛盾があ

#### RAG の実行

上記のプロンプトを利用して Amazon Bedrock の InvokeModel API を実行してみましょう。

In [44]:
def invoke_model(prompt, model_id="anthropic.claude-3-haiku-20240307-v1:0"):
    body = json.dumps({
        "max_tokens": 1024,
        "messages": [{"role": "user", "content": prompt}],
        "anthropic_version": "bedrock-2023-05-31",
        "temperature": 0,
    })

    try:
        response_bedrock = bedrock_runtime.invoke_model(
            body=body, modelId=model_id, accept=accept, contentType=content_type,
        )
        response_body = json.loads(response_bedrock.get("body").read())
        return response_body.get("content")[0]["text"]
    except ClientError as error:
        raise error

print(invoke_model(prompt, model_id))

以下の3つの検索クエリを提案します:

1. "Amazon Bedrock model evaluation" "機能" "capabilities"
2. "Amazon Bedrock" "model assessment" "評価" "features"
3. "Amazon Bedrock" "model" "評価" "performance" "analysis"

これらのクエリは、Amazon Bedrockのモデル評価機能に関する情報を幅広く検索できるよう設計されています。
日本語と英語を組み合わせることで、より包括的な検索結果が得られると考えられます。
各クエリは30トークン以内に収まっています。


#### プロンプトエンジニアリング実践

##### 1. 回答文に XML タグを含めないようにする

`<excerpts>` や `<excerpt>` タグはシステム内部で用いるものであり、ユーザーからするとよくわからない謎の文字列です。
このままではユーザーを不安にしてしまうので、回答生成に条件を追加し、回答に XML タグを含めないようにしましょう。

In [13]:
prompt = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
"""

for item in response_kendra["ResultItems"]:
    prompt += f"<excerpt>{item['Content']}</excerpt>\n"

prompt += f"""</excerpts>

<excerpts>タグ内の情報を参考にして、<query>タグ内のユーザーの質問に答えてください。

<query>
{query_text}
</query>
"""

rule = """
ただし、<rule>タグ内の回答ルールを遵守してください。

<rule>
- 回答には<excerpts>, <excerpt>, <query>タグを含めないこと。
</rule>
"""

print(prompt + rule)
print("-----")
print(invoke_model(prompt + rule, model_id))


あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
<excerpt>print( f"Finished generating text with your provisioned custom model {model_id}.") if __name__ == "__main__": main() モデルカスタマイズに関するガイドライン モデルをカスタマイズする理想的なパラメータは、データセットと、モデルが対象とするタスクに よって異なります。 値をいろいろ試して、どのパラメータがお客様自身のケースで最も適切に機能す るかを確認する必要があります。 参考までに、モデル評価ジョブを実行してモデルを評価してくださ い。 詳細については、「モデル評価」を参照してください。 このトピックでは、Amazon Titan Text G1 - Express モデルをカスタマイズするための基準となるガ イドラインと推奨値を示します。 他のモデルについては、プロバイダーのドキュメントを確認してく ださい。 ファインチューニングジョブの送信時に生成される出力ファイルに含まれるトレーニングと検証のメ トリクスを使用して、パラメータを調整します。 出力を書き込んだ Amazon S3 バケットでこれらの ファイルを検索するか、GetCustomModelオペレーションを使用してください。</excerpt>
<excerpt>AWSドキュメントAmazon Bedrockユーザーガイド Amazon Titan Text G1 - Express 翻訳は機械翻訳により提供されています。 提供された翻訳内容と英語版の間で齟齬、不一致または矛盾がある場合、英語版が優先します。 モデルカスタマイズに関するガイドライン モデルをカスタマイズする理想的なパラメータは、データセットと、モデルが対象とするタスクによって異なります。 値をいろいろ試して、どのパラメータがお客様自身のケースで最も適切に機能するかを確認する必要があります。 提供された翻訳内容と英語版の間で齟齬、不一致または矛盾があ

##### 2. 検索インデックスに含まれていない内容は答えないようにする

ここまでのプロンプトでは、Kendra に含まれていない知識に関する質問をされても LLM (Claude 3 Haiku) の持っている知識で答えようとします。

<!--例えば、Amazon SageMaker Studio の上でネイティブに利用できる開発環境は、JupyterLab、RStudio、SageMaker Canvas、Code Editor (OSS 版 Visual Studio Code)、SageMaker Studio Classic なのですが、RAG システムに問い合わせると間違った答え (ハルシネーション) が返ってきます。-->

In [17]:
query_text = "アマゾン川はどの国を通っていますか？"

# Kendra での検索 - 無関係なドキュメントが検索される
response_kendra = kendra.retrieve(
    QueryText=query_text,  # 検索クエリ
    IndexId=kendra_index_id,  # Kendra Index ID
    AttributeFilter={
        "EqualsTo": {
            "Key": "_language_code",
            "Value": {
                "StringValue": "ja",  # 日本語ドキュメントを検索 (default: en)
            }
        }
    },
    #PageNumber=1,
    PageSize=3,  # 一度に返すドキュメント抜粋の数 (default: 10)
)

# LLM へのプロンプト
prompt = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
"""

for item in response_kendra["ResultItems"]:
    prompt += f"<excerpt>{item['Content']}</excerpt>\n"  # 質問と無関係なドキュメントを追加

prompt += f"""</excerpts>

<excerpts>タグ内の情報を参考にして、<query>タグ内のユーザーの質問に答えてください。

<query>
{query_text}
</query>
"""

rule = """
ただし、<rule>タグ内の回答ルールを遵守してください。

<rule>
- 回答には<excerpts>, <excerpt>, <query>タグを含めないこと。
</rule>
"""

print(prompt + rule)
print("-----")

# RAG 実行
print(invoke_model(prompt + rule, model_id))


あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
<excerpt>各エンティティには、エンティティタイプが正しく検出されたという Amazon Comprehend の信頼レベルを示すスコアも含まれます。 スコアの低いエンティティを除外して、誤検出を使ってしまう行うリスクを減らすことができます。 以下の表はエンティティのタイプをまとめています。 型	説明 COMMERCIAL_ITEM ブランド製品 DATE 日付 (11/25/2017 など)、曜日 (Tuesday)、月 (May)、または時刻 (8:30 a.m.) EVENT フェスティバル、コンサート、選挙などのイベント LOCATION 国、都市、湖、建物などの特定の場所 ORGANIZATION 政府、企業、宗教、スポーツチームなどの大規模な組織 OTHER 他のどのエンティティカテゴリにも当てはまらないエンティティ PERSON 個人、グループ、ニックネーム、架空の人物 QUANTITY 通貨、パーセント、数値、バイト数などの数量。 TITLE 映画、本、歌など、あらゆる作品や創作作品に付けられた正式名称。 エンティティの検出オペレーションは、Amazon Comprehend がサポートする主要言語のいずれかを使用して実行できます。</excerpt>
<excerpt>• コンソールを使用したリアルタイム分析 • ターゲット感情の出力例 ターゲット感情 26 https://aws.amazon.com/blogs/machine-learning/extract-granular-sentiment-in-text-with-amazon-comprehend-targeted-sentiment/ https://aws.amazon.com/blogs/machine-learning/extract-granular-sentiment-in-text-with-amazon-comprehend-targeted-sentiment/ Amazon Comprehend 開発

今回は、ナレッジベース (Kendra) から関連するドキュメントが検索できなかった場合には無理に回答しないようにプロンプトを工夫してみましょう。

In [18]:
rule = """
ただし、<rule>タグ内の回答ルールを遵守してください。

<rule>
- 回答には<excerpts>, <excerpt>, <query>タグを含めないこと。
- 回答は<answer>タグで囲むこと。
- 回答する前に、質問へ回答するために必要な情報が与えられているか考え、<thinking>タグに判断の理由を書いてください。
</rule>
"""

print(prompt + rule)
print("-----")

# RAG 実行
output = invoke_model(prompt + rule, model_id)
print(output)
print("-----")

match = re.search(r"<answer>(.*?)</answer>", output, re.DOTALL)
if match:
    answer = match.group(1).strip()
    print(answer)



あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
<excerpt>各エンティティには、エンティティタイプが正しく検出されたという Amazon Comprehend の信頼レベルを示すスコアも含まれます。 スコアの低いエンティティを除外して、誤検出を使ってしまう行うリスクを減らすことができます。 以下の表はエンティティのタイプをまとめています。 型	説明 COMMERCIAL_ITEM ブランド製品 DATE 日付 (11/25/2017 など)、曜日 (Tuesday)、月 (May)、または時刻 (8:30 a.m.) EVENT フェスティバル、コンサート、選挙などのイベント LOCATION 国、都市、湖、建物などの特定の場所 ORGANIZATION 政府、企業、宗教、スポーツチームなどの大規模な組織 OTHER 他のどのエンティティカテゴリにも当てはまらないエンティティ PERSON 個人、グループ、ニックネーム、架空の人物 QUANTITY 通貨、パーセント、数値、バイト数などの数量。 TITLE 映画、本、歌など、あらゆる作品や創作作品に付けられた正式名称。 エンティティの検出オペレーションは、Amazon Comprehend がサポートする主要言語のいずれかを使用して実行できます。</excerpt>
<excerpt>• コンソールを使用したリアルタイム分析 • ターゲット感情の出力例 ターゲット感情 26 https://aws.amazon.com/blogs/machine-learning/extract-granular-sentiment-in-text-with-amazon-comprehend-targeted-sentiment/ https://aws.amazon.com/blogs/machine-learning/extract-granular-sentiment-in-text-with-amazon-comprehend-targeted-sentiment/ Amazon Comprehend 開発

この、最終的な答えを提供する前に Claude に考える時間を与える手法は「思考の連鎖 (Chain of Thoughts; CoT)」プロンプトと呼ばれます。
Claude の段階的な推論と最終的な応答を区別しやすくするために、`<thinking>` や `<answer>` タグを使うことが推奨されています。
詳しくは [Anthropic のプロンプトエンジニアリングガイド](https://docs.anthropic.com/ja/docs/let-claude-think#claude-2)を参照してください。

##### 3. 回答の根拠となる引用を付ける

本セクションの締めくくりとして、回答文だけでなく、回答の根拠となる文書を引用するようにしてみましょう。

まず、プロンプトに Kendra で検索したドキュメントの抜粋だけでなく、ドキュメント ID やタイトルも含めるようにします。

In [22]:
query_text = "Amazon Bedrockって何ですか？"

response_kendra = kendra.retrieve(
    QueryText=query_text,  # 検索クエリ
    IndexId=kendra_index_id,  # Kendra Index ID
    AttributeFilter={
        "EqualsTo": {
            "Key": "_language_code",
            "Value": {
                "StringValue": "ja",  # 日本語ドキュメントを検索 (default: en)
            }
        }
    },
    #PageNumber=1,
    PageSize=10,  # 一度に返すドキュメント抜粋の数 (default: 10)
)

prompt = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
"""

for item in response_kendra["ResultItems"]:
    prompt += "<excerpt>"
    prompt += f"<document_id>{item['DocumentId']}</document_id>"
    prompt += f"<title>{item['DocumentTitle']}</title>"
    prompt += f"<content>{item['Content']}</content>"
    prompt += "</excerpt>\n"


prompt += f"""</excerpts>

<excerpts>タグ内の情報を参考にして、<query>タグ内のユーザーの質問に答えてください。

<query>
{query_text}
</query>
"""

print(prompt)


あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
<excerpt><document_id>https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/agents-manage.html</document_id><title>Amazon Bedrock エージェントを管理する - Amazon Bedrock</title><content>AWS ドキュメントを使用するには、JavaScript を有効にする必要があります。 手順については、使用するブラウザのヘルプページを参照してください。 ドキュメントの表記規則 イベントのトレース アクショングループの管理 このページは役に立ちましたか? - はい ページが役に立ったことをお知らせいただき、ありがとうございます。 お時間がある場合は、何が良かったかお知らせください。 今後の参考にさせていただきます。 このページは役に立ちましたか? - はい ページが役に立ったことをお知らせいただき、ありがとうございます。 お時間がある場合は、何が良かったかお知らせください。 今後の参考にさせていただきます。 このページは役に立ちましたか? - いいえ このページは修正が必要なことをお知らせいただき、ありがとうございます。 ご期待に沿うことができず申し訳ありません。 お時間がある場合は、ドキュメントを改善する方法についてお知らせください。</content></excerpt>
<excerpt><document_id>https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/quotas.html</document_id><title>Amazon Bedrock のクォータ - Amazon Bedrock</title><content>ブラウザで JavaScript が無効になっているか、使用できません。 AWS ドキュメントを使用するには、JavaScript を有効にする必要があ

追加したドキュメント ID やタイトルの情報をもとに、回答の根拠となるドキュメントを引用するようにします。ここでは、LLM の出力を XML タグで構造化し、それをレンダリングする際にパースする方針を取ります。

In [23]:
rule = """
回答内で検索結果からの情報を参照する場合は、情報が見つかったソースのドキュメントへの引用を含める必要があります。
各結果には、参照すべき対応するdocument_idとtitleを付与します。
回答に複数のドキュメントの抜粋からの情報が含まれている場合、<sources>には複数の<source>が含まれる可能性があることに注意してください。
<excerpts>を回答で直接引用しないでください。あなたの仕事は、ユーザーの質問に可能な限り簡潔に答えることです。
以下の形式で回答を出力する必要があります。フォーマットとスペースに注意して、正確に従ってください。
<answer>
  <answer_part>
    <text>
      最初の回答テキスト
    </text>
    <sources>
      <source>
        <document_id>document_id</document_id>
        <title>title</title>
      </source>
    </sources>
  </answer_part>
  <answer_part>
    <text>
      2番目の回答テキスト
    </text>
    <sources>
      <source>
        <document_id>document_id</document_id>
        <title>title</title>
      </source>
    </sources>
  </answer_part>
</answer>
"""

In [24]:
print(prompt + rule)
print("-----")

# RAG 実行
output = invoke_model(prompt + rule, model_id)
print(output)
print("-----")


あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
<excerpt><document_id>https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/agents-manage.html</document_id><title>Amazon Bedrock エージェントを管理する - Amazon Bedrock</title><content>AWS ドキュメントを使用するには、JavaScript を有効にする必要があります。 手順については、使用するブラウザのヘルプページを参照してください。 ドキュメントの表記規則 イベントのトレース アクショングループの管理 このページは役に立ちましたか? - はい ページが役に立ったことをお知らせいただき、ありがとうございます。 お時間がある場合は、何が良かったかお知らせください。 今後の参考にさせていただきます。 このページは役に立ちましたか? - はい ページが役に立ったことをお知らせいただき、ありがとうございます。 お時間がある場合は、何が良かったかお知らせください。 今後の参考にさせていただきます。 このページは役に立ちましたか? - いいえ このページは修正が必要なことをお知らせいただき、ありがとうございます。 ご期待に沿うことができず申し訳ありません。 お時間がある場合は、ドキュメントを改善する方法についてお知らせください。</content></excerpt>
<excerpt><document_id>https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/quotas.html</document_id><title>Amazon Bedrock のクォータ - Amazon Bedrock</title><content>ブラウザで JavaScript が無効になっているか、使用できません。 AWS ドキュメントを使用するには、JavaScript を有効にする必要があ

正規表現を使って XML タグをパースします

In [28]:
def parse_answer(xml):
    # XMLデータから<answer_part>タグで囲まれた部分を抽出する
    answer_parts = re.findall(r'<answer_part>(.*?)</answer_part>', xml, re.DOTALL)
    result = ""  # 結果を格納する変数
    source_refs = ""  # ソース情報を格納する変数
    source_index = 1  # ソースに付与する連番
    unique_refs = set()  # 重複するソースのタイトルを避けるための集合
    source_dic = {}  # ソースのタイトルと番号のマッピング
    for part in answer_parts:
        # 回答の本文を抽出する
        text = re.search(r'<text>(.*?)</text>', part, re.DOTALL).group(1).strip()
        # ソース情報を抽出する
        sources = re.findall(r'<source>(.*?)</source>', part, re.DOTALL)
        source_str = ""  # 現在の部分のソース情報 (e.g. [1]) を格納する文字列
        for source in sources:
            document_id = re.search(r'<document_id>(.*?)</document_id>', source).group(1)
            title = re.search(r'<title>(.*?)</title>', source).group(1)
            # ソースのタイトルが重複する場合は、前に出現したものと同じ番号を付与する
            if title in unique_refs:
                source_str += f" \[{source_dic[title]}\]"
            else:
                source_str += f" \[{source_index}\]"
                unique_refs.add(title)
                source_dic[title] = source_index
                source_refs += f"\[{source_index}\] [{title}]({document_id})  \n"  # ソース情報を追加
                source_index += 1
        # 回答の本文とソース情報を結果に追加する
        result += f"{text}{source_str}\n\n"
    # 最後にソースの連番とタイトルを追加する
    return result + "\n\n" + source_refs


display(Markdown(parse_answer(output)))

Amazon Bedrock は、AWS が提供するマネージドサービスで、自然言語処理 (NLP) や対話型 AI アプリケーションの構築を容易にするものです。Amazon Bedrock を使用すると、開発者は事前に構築されたエージェントを使用して、対話型 AI アプリケーションを迅速に構築できます。 \[1\]



\[1\] [Amazon Bedrock とは - Amazon Bedrock](https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/what-is-bedrock.html)  


### Pre-retrieval: クエリ拡張

クエリ拡張は、単一のクエリを複数のクエリに拡張することで多様な検索結果を取得し、生成される回答の適合性を高めるための手法です。クエリ拡張にも、単純に複数の異なるクエリを作成するマルチクエリ (multi query) や、元の質問を分解して個々の質問に答えるためのクエリを作成するサブクエリ (sub query) などさまざまなアプローチがありますが、今回はシンプルなマルチクエリのアプローチを採用しています。LLM は generator と同様に Claude 3 Haiku を用いてクエリを拡張します。

必ずしもユーザーのクエリとソースドキュメントの表現が一致しているわけではありません。「ナレッジベース」と「Knowledge Bases」等、日本語と英語との表記揺れや、類義語、タイポなど、さまざまな要因で初期のクエリでは回答に必要な情報が十分に取得できないことがあります。クエリを拡張し、類似するキーワードを複数用いることで、キーワード検索とベクトル検索のハイブリッド検索のような、確実性と曖昧性を兼ね備えた検索が実現されることが期待されます。

In [52]:
n_queries = 3

prompt = f"""
検索エンジンに入力するクエリを最適化し、様々な角度から検索を行うことで、より適切で幅広い検索結果が得られるようにします。 
以下の<question>タグ内にはユーザーの入力した質問文が入ります。
この質問文に基づいて、{n_queries}個の検索用クエリを生成してください。
各クエリは30トークン以内とし、日本語と英語を適切に混ぜて使用することで、広範囲の文書が取得できるようにしてください。

<question>
{query_text}
<question>
"""

print(invoke_model(prompt))

以下の3つの検索クエリを提案します:

1. "Amazon Bedrock model evaluation" OR "Amazon Bedrock モデル評価"
2. "Amazon Bedrock model assessment" OR "Amazon Bedrock モデル評価機能"
3. "Amazon Bedrock model performance" OR "Amazon Bedrock モデル性能評価"

これらのクエリは、Amazon Bedrockのモデル評価機能に関する情報を幅広く検索できるよう、日本語と英語を組み合わせて設計しています。
モデル評価、モデル性能、モデル assessment といった関連キーワードを含むことで、より適切な検索結果が得られると考えられます。
各クエリは30トークン以内に収まっています。


このままだとプログラム的に扱いづらいので出力フォーマットを指定します。

In [53]:
output_format = ""
for i in range(n_queries):
    output_format += f"query {i+1}: fill a query here\n"

format = f"""
生成されたクエリは、<format>タグ内のフォーマットに従って出力してください。

<format>
{output_format}
</format>
"""

queries = invoke_model(prompt + output_format)
print(queries)

<question>
Amazon Bedrockのモデル評価機能ではどのようなことができますか？
</question>

query 1: Amazon Bedrock model evaluation capabilities 評価 機能 分析 性能 測定 検証 モデル 品質 改善
query 2: Amazon Bedrock モデル評価 機能 使用方法 ユースケース 機能概要 性能指標 モデル検証
query 3: Amazon Bedrock model assessment features 評価指標 モデル改善 model performance analysis 品質管理


あとは正規表現で抜き出してあげれば後段の Kendra への入力にすることができます。

In [54]:
queries = [query for query in queries.split("\n") if re.match(r'query \d+:[^:]+$', query)]
queries

['query 1: Amazon Bedrock model evaluation capabilities 評価 機能 分析 性能 測定 検証 モデル 品質 改善',
 'query 2: Amazon Bedrock モデル評価 機能 使用方法 ユースケース 機能概要 性能指標 モデル検証',
 'query 3: Amazon Bedrock model assessment features 評価指標 モデル改善 model performance analysis 品質管理']

### Post-retrieval: 検索結果の関連度評価

このステップでは検索結果が、元のユーザーからの質問に関連したものになっているかを評価します。検索で得られたドキュメントの抜粋 (チャンク) というのは必ずしも質問に回答するための情報を含んでいるとは限らず、誤った回答を誘発するような内容も含んでしまっていることがあります。例えば、“[Corrective Retrieval Augmented Generation](https://arxiv.org/abs/2401.15884)” \[Shi-Qi Yan et al. (2024)\] の論文で提案されている手法 (Corrective RAG; CRAG) では、検索結果の関連度を評価し、その評価結果をもとに最終回答を生成します。CRAG ではドキュメントの抜粋をそれぞれ Correct (正確) / Ambiguous (曖昧) / Incorrect (不正確) のカテゴリに分類しますが、今回は簡単のために関連しているか否かの yes / no の二値に分類します。LLM は他のステップと同様に Claude 3 Haiku を用います。

In [75]:
query_text = "Amazon Bedrock で Claude 3 Sonnet の基盤モデルに関する情報を取得する Python コードを教えて"
response_kendra = kendra.retrieve(
    QueryText=query_text,  # 検索クエリ
    IndexId=kendra_index_id,  # Kendra Index ID
    AttributeFilter={"EqualsTo": {"Key": "_language_code", "Value": {"StringValue": "ja",}}},
    PageSize=5,  # 一度に返すドキュメント抜粋の数 (default: 10)
)

result_items = []

for ii, item in enumerate(response_kendra["ResultItems"]):
    result_item = ""
    result_item += "-" * 30 + "\n"
    result_item += f"Item {ii+1}\n"
    result_item += f"DocumentTitle:{item['DocumentTitle']}\n"
    result_item += "Content\n"
    result_item += item["Content"] + "\n"
    result_items.append(result_item)

pprint(result_items)

['------------------------------\n'
 'Item 1\n'
 'DocumentTitle:基盤モデルに関する情報の取得 - Amazon Bedrock\n'
 'Content\n'
 'レスポンスは、プロビジョニングされたスループットグラフのベースモデル IDs またはベースモデル ID に含まれていないモデル ID も返します。 '
 'Amazon Bedrock ベースモデル ID (オンデマンドスループット)   IDs   これらのモデル IDs '
 'は、下位互換性のために廃止されました。 特定の基盤モデルに関する情報を返すには、モデル ID を指定して '
 'GetFoundationModelリクエストを送信します。 タブを選択すると、インターフェイスまたは言語でのコード例が表示されます。 AWS CLI '
 'Amazon Bedrock 基盤モデルを一覧表示します。 aws bedrock list-foundation-models v2 '
 'Anthropic Claude に関する情報を取得します。 aws bedrock get-foundation-model '
 '--model-identifier anthropic.claude-v2 Python Amazon Bedrock 基盤モデルを一覧表示します。 '
 "import boto3 bedrock = boto3.client(service_name='bedrock') "
 'bedrock.list_foundation_models() v2 Anthropic Claude に関する情報を取得します。 import '
 "boto3 bedrock = boto3.client(service_name='bedrock') "
 "bedrock.get_foundation_model(modelIdentifier='anthropic.claude-v2')\n",
 '------------------------------\n'
 'Item 2\n'
 'DocumentTitle:Amazon Bedrock でマルチモーダルプロンプトで Anthropic Claude 3 を呼び

In [77]:
for result_item in result_items:
    prompt = f"""
    あなたは、ユーザーからの質問と検索で得られたドキュメントの関連性を評価する専門家です。
    <excerpt>タグ内は、検索により取得したドキュメントの抜粋です。

    <excerpt>{result_item}</excerpt>
    
    <question>タグ内は、ユーザーからの質問です。

    <question>{query_text}</question>
    
    このドキュメントの抜粋は、ユーザーの質問に回答するための正確な情報を含んでいるかを慎重に判断してください。
    正確な情報を含んでいる場合は 'yes'、含んでいない場合は 'no' のバイナリスコアを返してください。
    バイナリスコアは<related></related>タグ内に格納してください。
    """

    print(invoke_model(prompt))
    print("-----")

<related>no</related>

この抜粋には、Amazon Bedrock で Claude 3 Sonnet の基盤モデルに関する情報を取得するためのコードは含まれていません。抜粋には、Anthropic Claude v2 モデルに関する情報を取得するコードが含まれていますが、Claude 3 Sonnet モデルに関する情報は含まれていません。
-----
<related>yes</related>

このドキュメントの抜粋には、Amazon Bedrock で Claude 3 Sonnet の基盤モデルを呼び出す Python コードの例が含まれています。具体的には、`invoke_claude_3_multimodal()` 関数の定義が示されており、この関数を使ってマルチモーダルプロンプトで Claude 3 を呼び出す方法が説明されています。したがって、この抜粋は、ユーザーの質問に回答するための正確な情報を含んでいると判断できます。
-----
<related>no</related>

このドキュメントの抜粋には、Amazon Bedrock で Claude 3 Sonnet の基盤モデルに関する情報は含まれていません。抜粋には、Anthropic Claude v2 モデルに関する情報が記載されていますが、Claude 3 Sonnet モデルに関する情報は見つかりませんでした。
-----
<related>no</related>

この抜粋には、Amazon Bedrock の Claude 3 Sonnet の基盤モデルに関する情報は含まれていません。抜粋には、Amazon Bedrock の基盤モデルの一覧表示や特定のモデルに関する情報の取得方法が記載されていますが、Claude 3 Sonnet のモデルに関する情報は見つかりません。したがって、この抜粋は、ユーザーの質問に回答するための正確な情報を含んでいないと判断しました。
-----
<related>no</related>

この抜粋には、Amazon Bedrock で Claude 3 Sonnet の基盤モデルに関する情報は含まれていません。この抜粋は、Amazon Bedrock でマルチモーダルプロンプトを使って Anthropic Claude

ここでは、Kendra の Retrieve API で得られたドキュメントの抜粋ひとつひとつに対して関連度評価を行っています。クエリ拡張と組み合わせた場合 (元のクエリ + 3つの拡張クエリ) は、4クエリ×10抜粋=40抜粋分、Claude 3 Haiku を呼び出すことになります。1抜粋当たり、プロンプト全体で800トークン前後になるため、一回の検索における関連度評価だけで30,000トークン程度消費するケースがあることになります。レイテンシーの影響も最小限に抑えるため、クエリ拡張と同様に非同期処理として実行するのが望ましいですが、運用の際には RPM (Requests processed per minute) や TPM (Tokens processed per minute) のクォータには注意してください。

## Naive RAG を LangChain で構築する

LangChain は、大規模言語モデル (LLM) を活用したアプリケーション開発のためのオープンソースフレームワークです。様々な言語モデル、データソース、API を組み合わせて、コンテキストを理解し推論できるアプリケーションを構築できます。プロンプトテンプレート、メモリ管理、モジュール化されたアーキテクチャなどの機能を備えており、チャットボット、質問応答システム、要約ツールなど、多様なユースケースに適用可能です。

langchain_aws は、LangChain と AWS を連携させる Python パッケージです。Amazon Bedrock、Amazon Kendra などのサービスと連携でき、AmazonKendraRetriever や ChatBedrock など AWS 固有のコンポーネントを提供しています。AmazonKendraRetriever は Amazon Kendra を利用したドキュメント検索、ChatBedrock は Amazon Bedrock の LLM を使ったチャットモデルです。

langchain_aws を使えば、AWS の豊富な機能と LangChain の柔軟性を組み合わせ、実用的で拡張性の高い自然言語処理アプリケーションを構築できます。LLM の能力をさらに引き出し、外部データと統合させた高度なアプリケーション開発が可能になります。

### 必要なモジュールのインポート

In [28]:
from langchain_aws import AmazonKendraRetriever
from langchain_aws import ChatBedrock
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
import langchain

langchain.verbose = True

### AmazonKendraRetriever

In [18]:
retriever = AmazonKendraRetriever(
    index_id=kendra_index_id,
    region_name="us-west-2",
    attribute_filter={"EqualsTo": {"Key": "_language_code", "Value": {"StringValue": "ja"}}},
    top_k=10,
)
retriever.invoke(input=query_text)

[Document(page_content='Document Title: モデルカスタマイズに関するガイドライン - Amazon Bedrock\nDocument Excerpt: \nAWSドキュメントAmazon Bedrockユーザーガイド Amazon Titan Text G1 - Express 翻訳は機械翻訳により提供されています。 提供された翻訳内容と英語版の間で齟齬、不一致または矛盾がある場合、英語版が優先します。 モデルカスタマイズに関するガイドライン モデルをカスタマイズする理想的なパラメータは、データセットと、モデルが対象とするタスクによって異なります。 値をいろいろ試して、どのパラメータがお客様自身のケースで最も適切に機能するかを確認する必要があります。 提供された翻訳内容と英語版の間で齟齬、不一致または矛盾がある場合、英語版が優先します。 モデルカスタマイズに関するガイドライン モデルをカスタマイズする理想的なパラメータは、データセットと、モデルが対象とするタスクによって異なります。 値をいろいろ試して、どのパラメータがお客様自身のケースで最も適切に機能するかを確認する必要があります。 参考までに、モデル評価ジョブを実行してモデルを評価してください。 詳細については、「モデル評価」を参照してください。 このトピックでは、Amazon Titan Text G1 - Express モデルをカスタマイズするための基準となるガイドラインと推奨値を示します。 他のモデルについては、プロバイダーのドキュメントを確認してください。 ファインチューニングジョブの送信時に生成される出力ファイルに含まれるトレーニングと検証のメトリクスを使用して、パラメータを調整します。\n', metadata={'result_id': '72259ad9-8f8e-4966-8d33-b1323981ee55-66e81669-dfcf-421c-a00c-ba3a07991a45', 'document_id': 's3://arag-workshop-kendra-04-809078683005/docs/20240525/bedrock/model-customization-guidelines.html', 'source': 'https://

### ChatBedrock

In [19]:

chat = ChatBedrock(
    model_id=model_id,
    region_name="us-west-2",
    model_kwargs={
        "temperature": 0,
        "max_tokens": 1024,
    }
)
chat.invoke(input=query_text)

AIMessage(content='Amazon Bedrockのモデル評価機能には以下のような主な機能があります:\n\n1. モデルの性能評価:\n   - 分類、回帰、クラスタリングなどのタスクに対するモデルの精度、再現率、F1スコアなどの指標を評価できます。\n   - 複数のモデルを比較して、最適なモデルを選択することができます。\n\n2. モデルのデバッグ:\n   - モデルの予測結果を詳細に分析し、エラーの原因を特定することができます。\n   - 特徴量の重要度を可視化したり、モデルの内部動作を理解するためのツールが提供されます。\n\n3. モデルのチューニング:\n   - ハイパーパラメータの最適化を自動で行うことができます。\n   - 様々な手法(グリッドサーチ、ランダムサーチ、ベイズ最適化など)を使ってチューニングを行えます。\n\n4. モデルのデプロイ:\n   - 評価済みのモデルをプロダクション環境にデプロイする機能が用意されています。\n   - モデルのバージョン管理や A/B テストなども行えます。\n\n5. モデルのモニタリング:\n   - デプロイ後のモデルの振る舞いを監視し、性能の劣化を検知することができます。\n   - 必要に応じてモデルの再トレーニングや更新を行えます。\n\nこのように、Amazon Bedrockのモデル評価機能は、モデルの開発から運用までのライフサイクル全体をサポートしています。', additional_kwargs={'usage': {'prompt_tokens': 31, 'completion_tokens': 492, 'total_tokens': 523}}, response_metadata={'model_id': 'anthropic.claude-3-haiku-20240307-v1:0', 'usage': {'prompt_tokens': 31, 'completion_tokens': 492, 'total_tokens': 523}}, id='run-05ac0bbb-22ba-4222-8fcb-57046673f38c-0')


### ChatPromptTemplate

In [25]:
system_template = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
{context}
</excerpts>

<excerpts>タグ内の情報を参考にして、ユーザーの質問に答えてください。
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("human", "{input}")
    ]
)

### Chain

ここまでは boto3 を直接叩くのと対して変わらないですが、LamgChain はその名の通り各コンポーネントをチェーンのように繋ぐことができるのが便利なところです。単純な RAG システムを構築するためのモジュールが用意されており、以下のようにして全体のフローをひとつのチェーンとして実行できます。

In [ ]:

question_answer_chain = create_stuff_documents_chain(chat, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": query_text})


#### 参考
LangChain において文書を要約したり質問に答えたりする際に、複数の文書をどのように処理するかはいくつか方式があります。主に以下の4つのタイプがあります。
1. stuff
- 全ての文書を1つのプロンプトに詰め込んで (stuff)、1回の API 呼び出しで処理する最もシンプルな方式。
- 文書が小さく数が少ない場合に適している。
- 大きな文書にはあまり向かない。
2. map_reduce
- 各文書に対して個別に LLM を適用し (map)、その結果を新しい文書とみなす。
- それらの新しい文書を別の文書結合チェーンに渡して単一の出力を得る (reduce)。
- 必要に応じて map された文書を再帰的に圧縮する。
- 大きな文書を扱うのに適している。
- API 呼び出し回数が多くなる。
3. refine
- 入力文書をループして反復的に回答を更新していく方式。
- 各文書について、現在の中間回答と共に LLM に渡し、新しい回答を得る。
- 1度に1つの文書しか LLM に渡さないため、多くの文書を分析するのに適している。
- stuff よりもはるかに多くの LLM 呼び出しが必要。
4. map_rerank
- 各文書に対してプロンプトを実行し、タスクを完了させるだけでなく、回答の確からしさもスコア付けさせる。
- スコアが最も高い回答を返す。
以上のように、扱う文書の量や目的に応じて適切な chain type を選択することで、LangChain を使った文書の要約や質問応答をより効果的に行うことができます。

https://github.com/langchain-ai/langchain/tree/master/libs/langchain/langchain/chains/combine_documents

## LangGraph

LangGraph は、LangChain をベースに構築された大規模言語モデルを用いたステートフルなマルチエージェントアプリケーションを開発するための Python ライブラリです。主な特徴は以下の通りです。

1. サイクル(循環)の実装が容易で、LLM をループで呼び出しながら次のアクションを決定するようなフローを簡単に実装できる。
2. グラフベースでエージェントのワークフローを定義し、グラフの状態を変更することで動的な応答が可能。
3. アプリケーション全体の状態を管理する GraphState を定義でき、状態管理が容易。
4. 複数の独立したエージェントを連携させて協調動作させることができる。
5. 永続化機能により、長時間実行のマルチセッションアプリケーションに対応。
6. LangChain エコシステムと統合されており、様々な機能を活用できる。

他にも、Advanced RAG のような複雑なフローをグラフベースのアーキテクチャによって簡単に実装できることもメリットです。LangGraph を用いると高度な自然言語処理アプリケーションをモジュール形式で容易に実装できます。

例えば、以下のようなフローを作る際、Retrieve や Grade などのノードをエッジ (矢印) で繋ぐことで状態 (state) 遷移の流れを構築します。
条件分岐も conditional edge と呼ばれるコンポーネントを利用することで簡単に実現できます。
![langgraph_crag](img/crag.png)
出典: [Corrective RAG (CRAG) - langchain-ai/langgraph](https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb)

<!--![langgraph_crag](img/crag.png)
出典: [Corrective RAG (CRAG) - langchain-ai/langgraph](https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb?ref=blog.langchain.dev)-->

### 各モジュールの構築

今回構築するグラフ構造は下図の通りです。質問が投げられたら `entry_point` を経由して、ひとつ目の条件分岐に入ります。`n_queries` パラメータが1以上であれば `generate_queries` ノードで `n_queries` 個のクエリ拡張を行い、`retrieve` ノードで検索を行います。ふたつ目の条件分岐では、`grade_documents_enabled` パラメータが `"Yes"` であれば `grade_documents` ノードを実行し、フィルタされたドキュメントの抜粋を元に `generate` ノードで回答生成を行います。

![graph](img/final_graph.png)

#### State

ノード間の状態 (state) は辞書形式で持つことにします。

In [5]:
from typing import Dict, TypedDict
from langchain_core.output_parsers import StrOutputParser


class GraphState(TypedDict):
    """Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]


#### Entry Point

はじめに条件分岐を置きたいため、グラフのエントリーポイントとしてはパススルーのノードをセットします。

In [18]:
def entry_point(state):
    """Pass through"""
    return state

#### ひとつ目の条件分岐

`n_queries` パラメータが1以上であれば `generate_queries_enable`、0以下であれば `generate_queries_not_enable` と判断するエッジのロジックを書きます。

![decide_to_generate_queries](img/decide_to_generate_queries.png)

In [19]:
def decide_to_generate_queries(state):
    """
    Determines whether to generate queries, or use the original query.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---DECIDE TO GENERATE QUERIES---")
    n_queries = state["keys"]["n_queries"]

    if n_queries > 0:
        print(
            "---DECISION: GENERATE QUERIES---"
        )
        return "generate_queries_enabled"
    else:
        print("---DECISION: NOT GENERATE QUERIES---")
        return "generate_queries_not_enabled"

#### Generate Queries: クエリ拡張

元のクエリ (`question`) に加えて `n_queries` のクエリを拡張するノードを定義します。

![generate_queries](img/generate_queries.png)

In [20]:
def generate_queries(state):
    """Generate a variety of queries (RAG-Fusion).

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): The updated graph state with generated queries.
    """
    print("---GENERATE QUERIES---")
    state_dict = state["keys"]
    question = state_dict["question"]
    n_queries = state_dict["n_queries"]

    llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        region_name="us-west-2",
        model_kwargs={
            "temperature": 0,
            "max_tokens": 512,
        }
    )

    output_format = ""
    for i in range(n_queries):
        output_format += f"{i+1}: fill a query here\n"

    prompt_template = """
検索エンジンに入力するクエリを最適化し、様々な角度から検索を行うことで、より適切で幅広い検索結果が得られるようにします。 
具体的には、類義語や日本語と英語の表記揺れを考慮し、多角的な視点からクエリを生成します。

以下の<question>タグ内にはユーザーの入力した質問文が入ります。
この質問文に基づいて、{n_queries}個の検索用クエリを生成してください。
各クエリは30トークン以内とし、日本語と英語を適切に混ぜて使用することで、広範囲の文書が取得できるようにしてください。

生成されたクエリは、<format>タグ内のフォーマットに従って出力してください。

<example>
question: Knowledge Bases for Amazon Bedrock ではどのベクトルデータベースを使えますか？
query 1: Knowledge Bases for Amazon Bedrock vector databases engine DB store
query 2: Amazon Bedrock ナレッジベース ベクトル ベクター エンジン データベース ストア インデックス
query 3: Amazon Bedrock RAG 検索拡張生成 埋め込み embedding ベクトル データベース エンジン
</example>

<format>
{output_format}
</format> 

<question>
{question}
</question>
""".format(
        output_format=output_format,
        n_queries=n_queries,
        question="{question}",  # as-is
    )
    prompt = ChatPromptTemplate.from_template(prompt_template)

    chain = prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))
    queries = chain.invoke({"question": question})
    queries = [query.replace('"', '') for query in queries]
    queries = [query for query in queries if re.match(r'^\d+:[^:]+$', query)]  # 数字:文字列の項目のみ抽出
    queries = [f"0: {question}"] + queries
    print(queries)

    state_dict["queries"] = queries

    return {"keys": state_dict}

#### Retrieve: Amazon Kendra を用いた検索

Amazon Kendra でクエリに基づいて検索するノードを定義します。

![retrieve](img/retrieve.png)

In [21]:
def retrieve(state):
    """Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    retriever = AmazonKendraRetriever(
        index_id=kendra_index_id,
        region_name="us-west-2",
        attribute_filter={"EqualsTo": {"Key": "_language_code", "Value": {"StringValue": "ja"}}},
        top_k=10
    )
    documents = []
    if "queries" in state_dict:
        queries = state_dict["queries"]
        for query in queries:
            documents.extend(retriever.invoke(query))
    else:
        documents = retriever.invoke(question)
    state_dict["documents"] = documents
    return {"keys": state_dict}

#### ふたつ目の条件分岐

`grade_documents_enabled` パラメータが `"Yes"` であれば `grade_documents_enabled` を返し、`"No"` であれば `grade_documents_not_enabled` を返すロジックを書きます。

![decide_to_grade_documents](img/decide_to_grade_documents.png)

In [22]:
def decide_to_grade_documents(state):
    """
    Determines whether to grade documents or not.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---DECIDE TO GRADE DOCUMENTS---")
    grade_documents_enabled = state["keys"]["grade_documents_enabled"]

    if grade_documents_enabled == "Yes":
        print(
            "---DECISION: GRADE DOCUMENTS---"
        )
        return "grade_documents_enabled"
    else:
        print("---DECISION: NOT GRADE DOCUMENTS---")
        return "grade_documents_not_enabled"

#### Grade Documents: 検索結果の関連度評価

Amazon Kendra の検索結果が元の質問 (`question`) に関連したものになっているかを判断し、関連しないようであれば除外するノードを定義する。

![grade_documents](img/grade_documents.png)

In [23]:
def grade_documents(state):
    """Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with relevant documents
    """

    print("---CHECK RELEVANCE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    llm = ChatBedrock(
        model_id=model_id,
        region_name="us-west-2",
        model_kwargs={
            "temperature": 0,
            "max_tokens": 128,
        }
    )

    system_template = """
あなたはユーザーからの質問と取得したドキュメントの関連性を評価します。

<excerpt>タグ内は、検索により取得したドキュメントの抜粋です。

<excerpt>
{context}
</excerpt>

ドキュメントの抜粋が、ユーザーの質問に関連するものであるかを判別してください。
関連していれば 'yes'、していなければ 'no' の binary score を返してください。
binary score は<related>タグに格納し、出力は <related>yes</related> もしくは <related>no</related> のみとしてください。
"""

    #parser = XMLOutputParser(tags=["related"])
    #print(parser.get_format_instructions())

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_template),
            ("human", "{question}")
        ]
    )
    

    chain = prompt | llm | StrOutputParser()

    filtered_docs = []

    for doc in documents:
        output = chain.invoke(
            {
                "question": question,
                "context": doc.page_content,
                #"format_instructions": parser.get_format_instructions(),
            }
        )
        pattern = r"<related>(.*?)</related>"
        match = re.search(pattern, output)

        if match:
            score = match.group(1)
            if score == "yes":
                print("---GRADE: DOCUMENT RELEVANT---")
                filtered_docs.append(doc)
            else:
                print("---GRADE: DOCUMENT NOT RELEVANT---")
                continue
        else:
            print("マッチする文字列がありません。")

    search = "Yes" if len(filtered_docs) == 0 else "No"

    state_dict["documents"] = filtered_docs
    state_dict["run_web_search"] = search

    return {
        "keys": state_dict
    }

#### Generate

検索で得られたドキュメントの抜粋をもとに質問への回答生成を行うノードを定義する。

![generate](img/generate.png)

In [24]:
def generate(state):
    """Generate documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    system_template = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
{context}
</excerpts>

<excerpts>タグ内の情報を参考にして、ユーザーの質問に答えてください。
ただし、<rule>タグ内の回答ルールを遵守してください。

<rule>
- 回答には<excerpts>, <excerpt>, <query>タグを含めないこと。
- 回答は<answer>タグで囲むこと。
- 回答する前に、質問へ回答するために必要な情報が与えられているか考え、<thinking>タグに判断の理由を書いてください。
</rule>
"""

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_template),
            ("human", "{question}")
        ]
    )

    llm = ChatBedrock(
        model_id=model_id,
        region_name="us-west-2",
        model_kwargs={
            "temperature": 0,
            "max_tokens": 1024,
        }
    )

    rag_chain = prompt | llm | StrOutputParser()

    output = rag_chain.invoke({"context": documents, "question": question})

    match = re.search(r"<answer>(.*?)</answer>", output, re.DOTALL)
    if match:
        generation = match.group(1).strip()
    else:
        generation = ""

    state_dict["generation"] = generation
    return {
        "keys": state_dict
    }
    

### Graph 全体を構築

![final_graph](img/final_graph.png)

In [25]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# ノードを追加
workflow.add_node("entry_point", entry_point)
workflow.add_node("generate_queries", generate_queries)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)

# グラフのフローを定義
workflow.set_entry_point("entry_point")
workflow.add_conditional_edges(
    "entry_point",
    decide_to_generate_queries,
    {
        "generate_queries_enabled": "generate_queries",
        "generate_queries_not_enabled": "retrieve",
    },
)
workflow.add_edge("generate_queries", "retrieve")
workflow.add_conditional_edges(
    "retrieve",
    decide_to_grade_documents,
    {
        "grade_documents_enabled": "grade_documents",
        "grade_documents_not_enabled": "generate",
    }
)
workflow.add_edge("grade_documents", "generate")
workflow.add_edge("generate", END)

# グラフのコンパイル
app = workflow.compile()

### Graph を実行

LangGraph で構築したグラフは `app.stream(inputs)` で実行できます。for loop を用いることでグラフ構造に従ってそれぞれのノードが実行されていきます。

In [29]:
def invoke_graph(app, inputs):
    for output in app.stream(inputs):
        for key, value in output.items():
            pprint(f"Node '{key}':")
        print("\n---\n")

    pprint(value["keys"]["generation"])

以下で `question` や `n_queries`、`grade_documents_enabled` の値を変えることで Advanced RAG の手法をオンオフしながら質問してみましょう。

<b> ＊ 今回の実装では負荷の軽減のため Kendra の検索や関連度評価は同期的に実行していますが、非同期的に並列で実行することでレイテンシーを大幅に削減することができます。 </b>

In [35]:

# 質問集
question = "Knowledge Bases for Amazon Bedrock ではどういったベクトルデータベースを利用できますか？"
#question = "Amazon Kendraを使ってWebサイトのコンテンツを検索可能にしたいと考えています。クロールの対象とするURLを制限する方法はありますか?"
#question = "Kendra で使用できるデータソースを全部教えて"
#question = "Amazon Kendra がサポートしているユーザーアクセス制御の方法は？"
#question = "Amazon Kendra の検索分析のメトリクスには何がありますか？"
#question = "Amazon Bedrock で Claude 3 Sonnet の基盤モデルに関する情報を取得する Python コードを教えて"
#question = "ナレッジベースでのembeddingモデルの選択肢は？"
#question = "Amazon Kendraで検索結果のランキングロジックをカスタマイズできますか？"
#question = "Amazon Kendraにはどんなエディションがありますか？"
#question = "Amazon Bedrock でモデルにアクセスするには何が必要ですか？"
#question = "Bedrockのagent機能は東京リージョンでは使えますか？"
#question = "外国語学習を効果的に進めるコツとは？"
#question = "Amazon CloudFrontの仕組みについて説明してください。"
#question = "Amazon EC2の特徴は何ですか？"

inputs = {"keys": {
    "question": question, 
    "n_queries": -1,  # generate_queries_not_enabled
    "grade_documents_enabled": "No",
    #"n_queries": 3,  # generate_queries_enabled
    #"grade_documents_enabled": "Yes",
}}

invoke_graph(app, inputs)

"Node 'entry_point':"

---

---DECIDE TO GENERATE QUERIES---
---DECISION: GENERATE QUERIES---
---GENERATE QUERIES---


['0: Knowledge Bases for Amazon Bedrock ではどういったベクトルデータベースを利用できますか？', '1: Amazon Bedrock knowledge base vector database engine storage', '2: Amazon Bedrock ナレッジベース ベクトルデータベース エンジン DB ストア', '3: Amazon Bedrock vector database engine index embedding']
"Node 'generate_queries':"

---

---RETRIEVE---
"Node 'retrieve':"

---

---DECIDE TO GRADE DOCUMENTS---
---DECISION: NOT GRADE DOCUMENTS---
---GENERATE---
"Node 'generate':"

---

"Node '__end__':"

---

('Knowledge Bases for Amazon Bedrock では、Amazon OpenSearch Serverless ベクターストアや '
 'Amazon Aurora データベースクラスターなどのベクトルデータベースを利用できます。\n'
 'ベクトルデータベースの設定時には、faissベクターインデックスがエンジンで設定されていることを確認する必要があります。nmslibベクトルインデックスがエンジンで設定されている場合は、別のベクトルインデックスを作成し、faissエンジンを選択する必要があります。')


## まとめ

本ノートブックでは、Amazon Bedrock と Amazon Kendra を用いて Naive RAG から Advanced RAG までを段階的に構築する方法について学びました。

今回の Naive RAG の構成は、Amazon Kendra でドキュメントを検索し、その検索結果を Amazon Bedrock の Claude モデルに渡して回答を生成するシンプルなアプローチです。
boto3 を直接使う方法と、LangChain を使ってコンポーネント化する方法を紹介しました。

Advanced RAGでは、いくつかの手法を組み合わせることで、より高度な検索と回答生成を実現しました。

- Pre-retrieval のクエリ拡張により、元のクエリだけでなく関連する複数のクエリで検索することで、より多様で適切な検索結果を得られるようにしました。
- Post-retrieval の関連度評価では、検索結果の各ドキュメントが元の質問に関連しているかを判定し、無関係なドキュメントを除外することで、回答の精度向上を図りました。

最後に、LangGraph というグラフベースのフレームワークを使って、Advanced RAG の一連の処理フローをモジュール化し、パラメータを切り替えるだけで簡単に実行できるようにしました。

本ノートブックで紹介したテクニックを応用することで、LLM とナレッジベース、検索エンジンを組み合わせた高度な質問応答システムを構築することができます。実務への適用の際は、非同期処理の導入などパフォーマンスチューニングも必要になるでしょう。

LLM の性能は目覚ましい発展を遂げていますが、すべての知識を言語モデル内に詰め込むのは現実的ではありません。社内の情報資産を安全に利活用しながら、言語モデルの汎用的な能力と組み合わせる本アプローチは、企業の生産性を高め、イノベーションを加速する有望な手段になると考えられます。ぜひ参考にしていただければ幸いです。